In [ ]:
!nvidia-smi

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# epoch 500 lr 0.001 patience 5-
from ultralytics import YOLO
 
# Load the model.
model = YOLO('yolov8s.pt')
 
# Training.
results = model.train(
   data='data.yaml',
   imgsz=640,
   epochs=500,
   lr0=0.001,
   patience=50,
   plots=True,
   name='imgs640-epoch500-lr0.001-patience50'
)

In [ ]:
import numpy as np

# Contoh confusion matrix (12x12)
confusion_matrix = np.array([
                            [0.48, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.20],
                            [0.05, 0.23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.45, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04, 0.0],
                            [0.0, 0.0, 0.0, 0.18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05],
                            [0.14, 0.0, 0.0, 0.0, 0.55, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05],
                            [0.0, 0.0, 0.0, 0.0, 0.0, 0.71, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.09, 0.0, 0.0, 0.95, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.71, 0.0, 0.0, 0.0, 0.0, 0.45],
                            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.94, 0.0, 0.0, 0.00, 0.00],
                            [0.0, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.95, 0.0, 0.0, 0.05],
                            [0.0, 0.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0., 0.0, 0.92, 0.0, 0.20],
                            [0.0, 0.0, 0.03, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0],
                            [0.33, 0.64, 0.52, 0.64, 0.45, 0.29, 0.05, 0.29, 0.06, 0.0, 0.08, 0.71, 0.0],
                            ])

# Ambil jumlah kelas objek
num_classes = confusion_matrix.shape[0]

# Inisialisasi variabel untuk menyimpan hasil evaluasi
accuracy_list = []
precision_list = []
recall_list = []
f1_score_list = []

# Loop untuk menghitung evaluasi untuk setiap kelas objek
for i in range(num_classes):
    TP = confusion_matrix[i, i]
    TN = confusion_matrix[:i, :i].sum() + confusion_matrix[:i, i+1:].sum() + confusion_matrix[i+1:, :i].sum() + confusion_matrix[i+1:, i+1:].sum()
    FP = confusion_matrix[:i, i].sum() + confusion_matrix[i+1:, i].sum()
    FN = confusion_matrix[i, :i].sum() + confusion_matrix[i, i+1:].sum()

    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    # Handling jika nilai precision atau recall adalah nol
    if precision == 0 or recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_score_list.append(f1_score)

# Hitung rata-rata dari hasil evaluasi
mean_accuracy = np.mean(accuracy_list)
mean_precision = np.mean(precision_list)
mean_recall = np.mean(recall_list)
mean_f1_score = np.mean(f1_score_list)

# Print hasil rata-rata evaluasi
print(f"Mean Accuracy: {mean_accuracy:.4f}")
print(f"Mean Precision: {mean_precision:.4f}")
print(f"Mean Recall: {mean_recall:.4f}")
print(f"Mean F1 Score: {mean_f1_score:.4f}")

#####################################################

# Fungsi untuk menghitung mAP50 dari confusion matrix
def calculate_map50(confusion_matrix):
    iou_threshold = 0.5
    num_classes = confusion_matrix.shape[0] - 1
    ap_list = []

    for i in range(num_classes):
        TP = confusion_matrix[i, i]
        FP = confusion_matrix[num_classes, i]
        FN = confusion_matrix[i, num_classes]

        precision = TP / (TP + FP)

#####################################################

# Fungsi untuk menghitung mAP50 dari confusion matrix
def calculate_map50(confusion_matrix):
    iou_threshold = 0.5
    num_classes = confusion_matrix.shape[0] - 1
    ap_list = []

    for i in range(num_classes):
        TP = confusion_matrix[i, i]
        FP = confusion_matrix[num_classes, i]
        FN = confusion_matrix[i, num_classes]

        precision = TP / (TP + FP)
        recall = TP / (TP + FN)

        iou = TP / (TP + FP + FN)

        if iou >= iou_threshold:
            ap_list.append(precision * recall)

    return np.mean(ap_list)

# Hitung mAP50
mAP50 = calculate_map50(confusion_matrix)
print(f"mAP50: {mAP50:.4f}")

In [ ]:
from ultralytics import YOLO
import cv2
import time

# Inisialisasi model YOLO
model = YOLO("Runs\\detect\\Train\\Weights\\best.pt")

# Tentukan sumber (0 untuk webcam, atau path ke gambar atau video)
results = model.predict(source=0 , show = True)
# source = 'path_to_image.jpg'  # Gunakan gambar tertentu
# source = 'path_to_video.mp4'  # Gunakan video tertentu

# Buka sumber (0 untuk webcam, atau path ke gambar atau video)
cap = cv2.VideoCapture(source)

# Variabel untuk perhitungan FPS dan timer
frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Lakukan prediksi pada frame
    results = model.predict(frame)

    # Hitung FPS
    frame_count += 1
    if frame_count % 30 == 0:  # Hitung FPS setiap 30 frame
        end_time = time.time()
        elapsed_time = end_time - start_time
        fps = frame_count / elapsed_time
        print(f"FPS: {fps:.2f}")
        frame_count = 0
        start_time = end_time

    if time.time() - start_time >= 5:  # Berhenti setelah 5 detik
        break

    if cv2.waitKey(1) & 0xFF == 27:  # Tekan tombol 'Esc' untuk keluar
        break

cap.release()
cv2.destroyAllWindows()
